# Define variables

In [61]:
# import pathlib

# image_path_dir = pathlib.Path("/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/animacoes/2d/")
# watermark_path = pathlib.Path("/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/animacoes/Logo final.png")
# output_path_dir = pathlib.Path("/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/animacoes/2d_watermark/")

# watermark_position = (30,30)
# watermark_scale = .25

# crop = (0, 150,0 ,37)

In [62]:
import pathlib
import os
import subprocess

watermark_path = pathlib.Path("/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/animacoes/Logo final.png")
root_path_dir = pathlib.Path("/mnt/disk01/prd-eng/consulting/036-FGEngenhariaPostoNO/animacoes/")
images_case = "3d"

watermark_position = (1200,725)
watermark_scale = .25

crop_margins = (0, 0,0,0)

gif_size_list = [600,800,1200,1800]
speed_multiplier_list = [0.5,0.75,1,1.25]

watermark_positioning_phase = False

# Adds watermarks

In [63]:
from PIL import Image
import pathlib

def add_watermark_to_image(
    image: Image, watermark: Image, watermark_position: tuple, 
) -> Image:
    image.paste(watermark, watermark_position, watermark)
    return image

def rescale_image(image: Image, image_scale: float) -> Image:
    new_width = int(image.width*watermark_scale)
    new_height = int(image.height*watermark_scale)
    image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    return image

def crop_image_margins(
    image: pathlib.Path, crop_margins: tuple
) -> Image:
    w, h =  image.width, image.height
    image = image.crop((crop_margins[0], crop_margins[1], w-crop_margins[2], h-crop_margins[3]))
    return image

In [ ]:
def process_image(image: Image, watermark: Image, watermark_position: tuple, crop_margins: tuple) -> Image:
    image = crop_image_margins(image=image, crop_margins=crop_margins)
    image = add_watermark_to_image(image=image, watermark=watermark, watermark_position=watermark_position)
    return image

In [64]:
def generate_gif_from_images(images_dir: pathlib.Path, gif_dir: pathlib.Path, image_size: int, speed_multiplier: float):
    gif_file = gif_dir/f"{images_dir.stem}_size{image_size}_speed{speed_multiplier}.gif"
    gif_file.unlink(missing_ok=True)
    command = f'ffmpeg -pattern_type glob -i "{images_dir}/*.png" \
        -filter_complex "setpts={1/speed_multiplier}*PTS,scale={image_size}:-1,split=2[s0][s1];\
            [s0]palettegen[p];\
            [s1][p]paletteuse=dither=bayer:bayer_scale=5" \
            {gif_file}'
    try:
        print(os.getcwd())
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

In [65]:
def convert_gif_to_mp4(gif_file_path: pathlib.Path, mp4_file_path:pathlib.Path):
    command = f'ffmpeg -i {gif_file_path} -movflags faststart \
        -pix_fmt yuv420p -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" \
        {mp4_file_path}'
    try:
        print(os.getcwd())
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

In [ ]:
watermark = Image.open(watermark_path)
watermark = rescale_image(image=watermark, image_scale=watermark_scale)

images_path_dir = root_path_dir/images_case
images_with_watermark_path_dir = root_path_dir/f"{images_case}_watermarked"
images_with_watermark_path_dir.mkdir(parents=True, exist_ok=True)

if watermark_positioning_phase:
    sample_file = Image.open(next(images_path_dir.iterdir()))
    process_image(image=sample_file, watermark=watermark, watermark_position=watermark_position, crop_margins=crop_margins)
else:
    # crop and add watermark
    for item in images_path_dir.iterdir():
        if item.is_dir():
            continue
        else:
            try:
                print(item.name)
                with Image.open(item) as img:
                    process_image(image=img, watermark=watermark, watermark_position=watermark_position, crop_margins=crop_margins)
                    img.save(images_with_watermark_path_dir/item.name)
            except IOError:
                continue
    
    # turn into gif
    gif_dir_path = root_path_dir/f"{images_case}_gifs"
    gif_dir_path.mkdir(parents=True, exist_ok=True)
    for image_size in gif_size_list:
        for speed_multiplier in speed_multiplier_list:
            generate_gif_from_images(images_dir=images_with_watermark_path_dir, 
                                     gif_dir=gif_dir_path, 
                                     image_size=image_size, 
                                     speed_multiplier=speed_multiplier
                                    )

    # turn gif into mp4
    for item in gif_dir_path.iterdir():
        if item.is_dir():
            continue
        elif item.suffix == ".gif":
            mp4_file=item.parent/(item.stem+".mp4")
            mp4_file.unlink(missing_ok=True)
            convert_gif_to_mp4(gif_file_path=item, mp4_file_path=mp4_file)